In [1]:
%pip install pyhwp

Note: you may need to restart the kernel to use updated packages.


In [21]:
%pip install python-dotenv

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [ ]:
# projectmission2/notebooks/data_preprocessing_bjs(all,hwp5proc).ipynb
import os
import subprocess
from bs4 import BeautifulSoup
import re
from typing import List, Dict, Any
import io
import contextlib
import glob
import json
import chromadb
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
from dotenv import load_dotenv
from openai import OpenAI

In [5]:
def convert_all_hwps_in_folder(input_directory, output_directory):
    """
    지정된 디렉터리 내의 모든 HWP 파일을 외부 hwp5proc 명령어로 XHTML로 변환합니다.

    Args:
        input_directory (str): HWP 파일들이 있는 디렉터리 경로.
        output_directory (str): 변환된 XHTML 파일을 저장할 디렉터리.
    """
    # 1. 출력 디렉터리 생성 (없을 경우)
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"출력 디렉터리가 없어 생성했습니다: {output_directory}")

    # 2. hwp5proc 실행 파일 경로 설정 (사용자 환경에 맞게 수정 필요)
    hwp5proc_executable = "/home/spai0323/myenv/bin/hwp5proc" # <-- 이 경로를 실제 사용하시는 경로로 수정해주세요.

    print(f"HWP 파일 일괄 변환 시작 (입력 폴더: {input_directory})")

    converted_count = 0
    error_files = []

    # 3. 입력 디렉터리 내 모든 파일 순회
    for filename in os.listdir(input_directory):
        if filename.lower().endswith(".hwp"): # .hwp 파일만 처리 (대소문자 구분 없음)
            input_path = os.path.join(input_directory, filename)
            base_name, _ = os.path.splitext(filename)
            output_path = os.path.join(output_directory, f'{base_name}.xhtml')

            # 4. hwp5proc 명령어 구성
            command = [
                hwp5proc_executable,
                "xml",  # hwp5proc의 xml 하위 명령어 사용
                input_path
            ]

            print(f"\n--- 변환 중: {filename} ---")
            try:
                # 5. subprocess.run으로 명령어 실행
                result = subprocess.run(
                    command,
                    capture_output=True,
                    text=True,
                    check=True
                )

                # 6. 캡처된 출력을 파일에 저장
                with open(output_path, 'w', encoding='utf-8') as f:
                    f.write(result.stdout)

                print(f"✅ 성공: {filename} -> {output_path}")
                converted_count += 1

            except subprocess.CalledProcessError as e:
                print(f"❌ 오류 발생 ({filename}):")
                print(f"   명령어: {' '.join(e.cmd)}")
                print(f"   반환 코드: {e.returncode}")
                # 오류가 발생한 파일은 따로 기록
                error_files.append({
                    "filename": filename,
                    "error": f"Return code: {e.returncode}, Stderr: {e.stderr.strip()}"
                })
            except FileNotFoundError:
                print(f"❌ 오류: hwp5proc 실행 파일을 찾을 수 없습니다. '{hwp5proc_executable}' 경로를 확인해주세요.")
                # hwp5proc 실행 파일이 없는 경우, 전체 작업 중단
                return
            except Exception as e:
                print(f"❌ 예기치 않은 오류 발생 ({filename}): {e}")
                error_files.append({
                    "filename": filename,
                    "error": str(e)
                })

    print("\n--- 변환 작업 완료 ---")
    print(f"총 변환된 파일 수: {converted_count}")

    if error_files:
        print(f"오류가 발생한 파일 수: {len(error_files)}")
        print("오류가 발생한 파일 목록:")
        for error_info in error_files:
            print(f"- {error_info['filename']}: {error_info['error']}")
    else:
        print("모든 HWP 파일 변환에 성공했습니다.")

# --- 실행 부분 ---
# HWP 파일이 있는 입력 폴더 경로
input_folder = '../data/raw/files'
# 변환된 XHTML 파일을 저장할 출력 디렉터리
output_directory_batch = '../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml'

# 함수 호출
convert_all_hwps_in_folder(input_folder, output_directory_batch)

HWP 파일 일괄 변환 시작 (입력 폴더: ../data/raw/files)

--- 변환 중: 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp ---
✅ 성공: 한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.hwp -> ../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml/한국연구재단_2024년 기초학문자료센터 시스템 운영 및 연구성과물 DB구.xhtml

--- 변환 중: 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .hwp ---
✅ 성공: 케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .hwp -> ../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml/케빈랩 주식회사_평택시 강소형 스마트시티 AI 기반의 영상감시 시스템 .xhtml

--- 변환 중: 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.hwp ---
✅ 성공: 수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.hwp -> ../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml/수협중앙회_수협중앙회 수산물사이버직매장 시스템 재구축 ISMP 수립 입.xhtml

--- 변환 중: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.hwp ---
✅ 성공: 국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.hwp -> ../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml/국립중앙의료원_(긴급)「2024년도 차세대 응급의료 상황관리시스템 구축.xhtml

--- 변환 중: 조선대학교_(재공고)2024 조선대학교 SW중심대학 사업관리시스템(WeHub) 구.hwp ---
✅ 성공: 조선대학교_(재공고)2024

In [6]:
def parse_hwp5proc_xhtml(file_path):
    """
    hwp5proc으로 변환된 XHTML 파일을 파싱하여 구조화된 데이터로 변환
    
    Args:
        file_path (str): XHTML 파일 경로
    
    Returns:
        dict: 파싱된 섹션과 표 데이터
    """
    print(f"XHTML 파일 파싱 시작: {file_path}")
    
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()
    except UnicodeDecodeError:
        # UTF-8로 읽기 실패 시 다른 인코딩 시도
        with open(file_path, 'r', encoding='cp949') as f:
            content = f.read()
    
    soup = BeautifulSoup(content, 'xml')  # XML 파서 사용
    
    # 결과 저장용 딕셔너리
    parsed_data = {
        'sections': [],
        'tables': [],
        'metadata': {
            'total_paragraphs': 0,
            'total_tables': 0,
            'file_size': len(content)
        }
    }
    
    # 1. 표(TableControl) 추출
    tables = soup.find_all('TableControl')
    print(f"발견된 표 개수: {len(tables)}")
    
    for i, table in enumerate(tables):
        table_data = extract_table_structure(table, table_id=i)
        if table_data:
            parsed_data['tables'].append(table_data)
    
    # 2. 일반 텍스트 문단 추출
    paragraphs = soup.find_all('Paragraph')
    print(f"발견된 문단 개수: {len(paragraphs)}")
    
    for i, para in enumerate(paragraphs):
        text_content = extract_paragraph_text(para, para_id=i)
        if text_content and text_content.get('content', '').strip():
            parsed_data['sections'].append(text_content)
    
    # 메타데이터 업데이트
    parsed_data['metadata']['total_paragraphs'] = len(parsed_data['sections'])
    parsed_data['metadata']['total_tables'] = len(parsed_data['tables'])
    
    print(f"파싱 완료 - 문단: {len(parsed_data['sections'])}, 표: {len(parsed_data['tables'])}")
    return parsed_data

def extract_table_structure(table_element, table_id):
    """
    TableControl 요소에서 표 데이터 추출
    """
    try:
        table_data = {
            'id': table_id,
            'type': 'table',
            'rows': [],
            'raw_content': '',
            'metadata': {}
        }
        
        # TableBody 찾기
        table_body = table_element.find('TableBody')
        if not table_body:
            return None
            
        # TableRow들 찾기
        rows = table_body.find_all('TableRow')
        
        for row in rows:
            row_data = []
            cells = row.find_all('TableCell')
            
            for cell in cells:
                # 셀 내의 Text 요소들 추출
                cell_text = ""
                text_elements = cell.find_all('Text')
                for text_elem in text_elements:
                    if text_elem.string:
                        cell_text += text_elem.string.strip() + " "
                
                row_data.append(cell_text.strip())
            
            if any(row_data):  # 빈 행이 아닌 경우만 추가
                table_data['rows'].append(row_data)
        
        # 표 내용을 자연어로 변환
        table_data['raw_content'] = convert_table_to_natural_language(table_data['rows'])
        
        return table_data
        
    except Exception as e:
        print(f"표 추출 중 오류 발생: {e}")
        return None

def extract_paragraph_text(paragraph_element, para_id):
    """
    Paragraph 요소에서 텍스트 추출
    """
    try:
        text_content = ""
        
        # LineSeg 요소들 찾기
        line_segs = paragraph_element.find_all('LineSeg')
        
        for line_seg in line_segs:
            # Text 요소들 추출
            text_elements = line_seg.find_all('Text')
            for text_elem in text_elements:
                if text_elem.string:
                    text_content += text_elem.string.strip() + " "
        
        return {
            'id': para_id,
            'type': 'paragraph',
            'content': text_content.strip(),
            'metadata': {}
        }
        
    except Exception as e:
        print(f"문단 추출 중 오류 발생: {e}")
        return None

def convert_table_to_natural_language(table_rows):
    """
    표 데이터를 자연어 형태로 변환
    """
    if not table_rows or len(table_rows) < 2:
        return ""
    
    # 첫 번째 행을 헤더로 가정
    headers = table_rows[0]
    content_rows = table_rows[1:]
    
    natural_text = []
    
    # 추진일정표 특별 처리
    if any('월' in str(header) for header in headers) and '구분' in str(headers):
        natural_text.append("프로젝트 추진일정은 다음과 같습니다:")
        
        for row in content_rows:
            if len(row) > 1 and row[0].strip():  # 첫 번째 컬럼이 비어있지 않은 경우
                task = row[0].strip()
                # 각 월별 일정 정보 추출
                schedule_info = []
                for i, cell in enumerate(row[1:], 1):
                    if cell.strip() and i < len(headers):
                        month = headers[i]
                        schedule_info.append(f"{month}에 {task}")
                
                if schedule_info:
                    natural_text.extend(schedule_info)
    else:
        # 일반 표 처리
        for row in content_rows:
            if len(row) >= len(headers):
                row_text = []
                for i, (header, cell) in enumerate(zip(headers, row)):
                    if cell.strip():
                        row_text.append(f"{header}: {cell.strip()}")
                
                if row_text:
                    natural_text.append(", ".join(row_text))
    
    return ". ".join(natural_text)

# 실제 파일로 테스트 실행
def main():
    # 실제 경로
    xhtml_file_path = '../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml/(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.xhtml'
    
    
    # 파일 존재 확인
    if not os.path.exists(xhtml_file_path):
        print(f"파일을 찾을 수 없습니다: {xhtml_file_path}")
        return
    
    # 파싱 실행
    parsed_result = parse_hwp5proc_xhtml(xhtml_file_path)
    
    # 결과 출력
    print("\n=== 파싱 결과 요약 ===")
    print(f"추출된 문단 수: {len(parsed_result['sections'])}")
    print(f"추출된 표 수: {len(parsed_result['tables'])}")
    print(f"파일 크기: {parsed_result['metadata']['file_size']:,} bytes")
    
    # 첫 번째 표 샘플 출력 (추진일정표)
    if parsed_result['tables']:
        print("\n=== 첫 번째 표 샘플 ===")
        first_table = parsed_result['tables'][0]
        print(f"표 ID: {first_table['id']}")
        print(f"행 수: {len(first_table['rows'])}")
        if first_table['raw_content']:
            print("자연어 변환 결과:")
            print(first_table['raw_content'][:500] + "..." if len(first_table['raw_content']) > 500 else first_table['raw_content'])
    
    # 첫 번째 문단 샘플 출력
    if parsed_result['sections']:
        print("\n=== 첫 번째 문단 샘플 ===")
        first_section = parsed_result['sections'][0]
        print(f"문단 ID: {first_section['id']}")
        content = first_section['content']
        print("내용:", content[:200] + "..." if len(content) > 200 else content)
    
    return parsed_result

# 실행
if __name__ == "__main__":
    result = main()

XHTML 파일 파싱 시작: ../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml/(재)예술경영지원센터_통합 정보시스템 구축 사전 컨설팅.xhtml
발견된 표 개수: 95
발견된 문단 개수: 3161
파싱 완료 - 문단: 2007, 표: 95

=== 파싱 결과 요약 ===
추출된 문단 수: 2007
추출된 표 수: 95
파일 크기: 3,926,663 bytes

=== 첫 번째 표 샘플 ===
표 ID: 0
행 수: 2
자연어 변환 결과:
사 업 명: 주관기관, 통합 정보시스템 구축 사전 컨설팅: 재단법인 예술경영지원센터

=== 첫 번째 문단 샘플 ===
문단 ID: 5
내용: 제 안 요 청 서


In [7]:
def create_structured_chunks(parsed_data: Dict[str, Any], max_chunk_size: int = 1000) -> List[Dict[str, Any]]:
    """
    파싱된 데이터를 구조화된 청크로 변환
    
    Args:
        parsed_data: parse_hwp5proc_xhtml 함수의 결과
        max_chunk_size: 최대 청크 크기 (토큰 수 기준)
    
    Returns:
        List[Dict]: 구조화된 청크들의 리스트
    """
    chunks = []
    
    print("청킹 작업 시작...")
    
    # 1. 표 데이터 청킹
    for table in parsed_data['tables']:
        table_chunk = create_table_chunk(table)
        if table_chunk:
            chunks.append(table_chunk)
    
    # 2. 문단 데이터 청킹
    paragraph_chunks = create_paragraph_chunks(parsed_data['sections'], max_chunk_size)
    chunks.extend(paragraph_chunks)
    
    # 3. 청크에 고유 ID 부여
    for i, chunk in enumerate(chunks):
        chunk['chunk_id'] = f"chunk_{i:04d}"
    
    print(f"청킹 완료 - 총 {len(chunks)}개 청크 생성")
    return chunks

def create_table_chunk(table_data: Dict[str, Any]) -> Dict[str, Any]:
    """
    표 데이터를 청크로 변환
    """
    if not table_data.get('raw_content') or not table_data['raw_content'].strip():
        return None
    
    # 표 유형 판단
    table_type = identify_table_type(table_data)
    
    chunk = {
        'type': 'table',
        'subtype': table_type,
        'content': table_data['raw_content'],
        'original_rows': len(table_data.get('rows', [])),
        'metadata': {
            'table_id': table_data.get('id'),
            'source': 'table_extraction',
            'priority': get_table_priority(table_type)
        }
    }
    
    return chunk

def identify_table_type(table_data: Dict[str, Any]) -> str:
    """
    표의 유형을 식별 (일정표, 예산표, 일반표 등)
    """
    content = table_data.get('raw_content', '').lower()
    rows = table_data.get('rows', [])
    
    # 헤더 행 확인
    if rows:
        header = ' '.join(rows[0]).lower()
        
        # 일정표 판단
        if any(keyword in header for keyword in ['월', '일정', '추진']):
            return 'schedule'
        
        # 예산표 판단
        if any(keyword in header for keyword in ['예산', '비용', '금액', '원']):
            return 'budget'
        
        # 조직표 판단
        if any(keyword in header for keyword in ['담당', '역할', '조직']):
            return 'organization'
    
    # 내용 기반 판단
    if any(keyword in content for keyword in ['일정', '월', '추진']):
        return 'schedule'
    elif any(keyword in content for keyword in ['예산', '비용', '천원', '만원', '십만원']):
        return 'budget'
    else:
        return 'general'

def get_table_priority(table_type: str) -> int:
    """
    표 유형별 우선순위 설정
    """
    priority_map = {
        'schedule': 10,  # 일정표는 높은 우선순위
        'budget': 8,
        'organization': 6,
        'general': 4
    }
    return priority_map.get(table_type, 4)

def create_paragraph_chunks(sections: List[Dict[str, Any]], max_chunk_size: int) -> List[Dict[str, Any]]:
    """
    문단 데이터를 적절한 크기의 청크로 분할
    """
    chunks = []
    current_chunk = []
    current_size = 0
    
    for section in sections:
        content = section.get('content', '')
        if not content.strip():
            continue
        
        # 대략적인 토큰 수 계산 (한국어: 글자수 * 0.7)
        estimated_tokens = len(content) * 0.7
        
        # 현재 청크에 추가할 수 있는지 확인
        if current_size + estimated_tokens <= max_chunk_size:
            current_chunk.append(content)
            current_size += estimated_tokens
        else:
            # 현재 청크 저장하고 새 청크 시작
            if current_chunk:
                chunks.append(create_paragraph_chunk(current_chunk))
            
            # 새 청크 시작
            if estimated_tokens > max_chunk_size:
                # 너무 큰 문단은 분할
                split_chunks = split_large_paragraph(content, max_chunk_size)
                chunks.extend(split_chunks)
                current_chunk = []
                current_size = 0
            else:
                current_chunk = [content]
                current_size = estimated_tokens
    
    # 마지막 청크 처리
    if current_chunk:
        chunks.append(create_paragraph_chunk(current_chunk))
    
    return chunks

def create_paragraph_chunk(content_list: List[str]) -> Dict[str, Any]:
    """
    문단 리스트를 하나의 청크로 변환
    """
    combined_content = ' '.join(content_list)
    
    # 문단 유형 판단
    paragraph_type = identify_paragraph_type(combined_content)
    
    chunk = {
        'type': 'paragraph',
        'subtype': paragraph_type,
        'content': combined_content,
        'paragraph_count': len(content_list),
        'metadata': {
            'source': 'paragraph_extraction',
            'priority': get_paragraph_priority(paragraph_type)
        }
    }
    
    return chunk

def identify_paragraph_type(content: str) -> str:
    """
    문단의 유형을 식별
    """
    content_lower = content.lower()
    
    # 제목/헤더 판단
    if any(keyword in content_lower for keyword in ['제', '장', '절', '항']):
        return 'header'
    
    # 목적/개요 판단
    if any(keyword in content_lower for keyword in ['목적', '개요', '배경']):
        return 'overview'
    
    # 결론/요약 판단
    if any(keyword in content_lower for keyword in ['결론', '요약', '종합']):
        return 'conclusion'
    
    # 방법론 판단
    if any(keyword in content_lower for keyword in ['방법', '절차', '과정']):
        return 'methodology'
    
    return 'general'

def get_paragraph_priority(paragraph_type: str) -> int:
    """
    문단 유형별 우선순위 설정
    """
    priority_map = {
        'overview': 9,
        'conclusion': 8,
        'methodology': 7,
        'header': 6,
        'general': 5
    }
    return priority_map.get(paragraph_type, 5)

def split_large_paragraph(content: str, max_chunk_size: int) -> List[Dict[str, Any]]:
    """
    큰 문단을 여러 청크로 분할
    """
    chunks = []
    sentences = re.split(r'[.!?]\s+', content)
    
    current_chunk = []
    current_size = 0
    
    for sentence in sentences:
        estimated_tokens = len(sentence) * 0.7
        
        if current_size + estimated_tokens <= max_chunk_size:
            current_chunk.append(sentence)
            current_size += estimated_tokens
        else:
            if current_chunk:
                chunk_content = '. '.join(current_chunk) + '.'
                chunks.append({
                    'type': 'paragraph',
                    'subtype': 'split_general',
                    'content': chunk_content,
                    'paragraph_count': 1,
                    'metadata': {
                        'source': 'paragraph_split',
                        'priority': 5
                    }
                })
            
            current_chunk = [sentence]
            current_size = estimated_tokens
    
    # 마지막 청크
    if current_chunk:
        chunk_content = '. '.join(current_chunk) + '.'
        chunks.append({
            'type': 'paragraph',
            'subtype': 'split_general',
            'content': chunk_content,
            'paragraph_count': 1,
            'metadata': {
                'source': 'paragraph_split',
                'priority': 5
            }
        })
    
    return chunks

def analyze_chunks(chunks: List[Dict[str, Any]]) -> None:
    """
    생성된 청크들을 분석하고 통계를 출력
    """
    print("\n=== 청크 분석 결과 ===")
    
    # 타입별 통계
    type_counts = {}
    subtype_counts = {}
    
    for chunk in chunks:
        chunk_type = chunk['type']
        chunk_subtype = chunk['subtype']
        
        type_counts[chunk_type] = type_counts.get(chunk_type, 0) + 1
        subtype_counts[chunk_subtype] = subtype_counts.get(chunk_subtype, 0) + 1
    
    print("타입별 분포:")
    for chunk_type, count in type_counts.items():
        print(f"  {chunk_type}: {count}")
    
    print("\n세부 타입별 분포:")
    for subtype, count in subtype_counts.items():
        print(f"  {subtype}: {count}")
    
    # 우선순위별 분포
    priority_counts = {}
    for chunk in chunks:
        priority = chunk['metadata']['priority']
        priority_counts[priority] = priority_counts.get(priority, 0) + 1
    
    print("\n우선순위별 분포:")
    for priority in sorted(priority_counts.keys(), reverse=True):
        print(f"  우선순위 {priority}: {priority_counts[priority]}개")

# 실행 함수
def main_chunking(parsed_result):
    """
    파싱 결과를 받아서 청킹 실행
    """
    chunks = create_structured_chunks(parsed_result, max_chunk_size=1000)
    
    # 분석 결과 출력
    analyze_chunks(chunks)
    
    # 샘플 청크 출력
    print("\n=== 샘플 청크 ===")
    
    # 일정표 청크 찾기
    schedule_chunks = [c for c in chunks if c.get('subtype') == 'schedule']
    if schedule_chunks:
        print("일정표 청크:")
        sample_schedule = schedule_chunks[0]
        print(f"  타입: {sample_schedule['type']}-{sample_schedule['subtype']}")
        print(f"  내용: {sample_schedule['content'][:200]}...")
        print(f"  우선순위: {sample_schedule['metadata']['priority']}")
    
    # 일반 문단 청크
    paragraph_chunks = [c for c in chunks if c['type'] == 'paragraph']
    if paragraph_chunks:
        print("\n문단 청크:")
        sample_paragraph = paragraph_chunks[0]
        print(f"  타입: {sample_paragraph['type']}-{sample_paragraph['subtype']}")
        print(f"  내용: {sample_paragraph['content'][:200]}...")
        print(f"  문단 수: {sample_paragraph['paragraph_count']}")
    
    return chunks

chunks = main_chunking(result)

청킹 작업 시작...
청킹 완료 - 총 145개 청크 생성

=== 청크 분석 결과 ===
타입별 분포:
  table: 65
  paragraph: 80

세부 타입별 분포:
  general: 49
  budget: 2
  schedule: 15
  header: 62
  split_general: 17

우선순위별 분포:
  우선순위 10: 15개
  우선순위 8: 2개
  우선순위 6: 62개
  우선순위 5: 18개
  우선순위 4: 48개

=== 샘플 청크 ===
일정표 청크:
  타입: table-schedule
  내용: 구  분: 사업 주최 / 주관 / 발주, 주요 업무 내용: 사업총괄 관리 및 시행 사업계획서, 제안요청서 작성 및 주관사업자 선정 과제수행 관리 및 검사, 결과 평가. 구  분: 협조부서, 주요 업무 내용: 업체 선정 및 계약 진행 계약 관련 업무 진행 및 협조. 구  분: 사업자문, 주요 업무 내용: 미술시장 자문. 구  분: 연구팀, 주요 업무 내용: ...
  우선순위: 10

문단 청크:
  타입: paragraph-header
  내용: 제 안 요 청 서 사 업 명 통합 정보시스템 구축 사전 컨설팅 주관기관 재단법인 예술경영지원센터 사 업 명 통합 정보시스템 구축 사전 컨설팅 주관기관 재단법인 예술경영지원센터 사 업 명 통합 정보시스템 구축 사전 컨설팅 주관기관 재단법인 예술경영지원센터 2024년 04월 구 분 소 속 전화번호 이메일 입찰관련 경영지원팀 02-708-2212 ksj37@go...
  문단 수: 64


In [8]:
def process_all_xhtml_files(directory_path):
    """
    지정된 디렉토리의 모든 XHTML 파일을 순회하며 파싱 및 청킹을 실행합니다.

    Args:
        directory_path (str): XHTML 파일들이 있는 디렉토리 경로
    """
    print(f"디렉토리 내 모든 XHTML 파일 처리 시작: {directory_path}")
    
    # 디렉토리 내 모든 .xhtml 파일 경로를 가져옵니다.
    file_paths = glob.glob(os.path.join(directory_path, '*.xhtml'))
    
    if not file_paths:
        print(f"경로에 .xhtml 파일이 없습니다: {directory_path}")
        return

    total_files = len(file_paths)
    print(f"총 {total_files}개의 파일이 발견되었습니다.")
    
    all_chunks = {}

    for i, file_path in enumerate(file_paths):
        print("\n" + "="*50)
        print(f"[{i+1}/{total_files}] 파일 처리 중: {os.path.basename(file_path)}")
        print("="*50)
        
        try:
            # 출력 임시 차단
            with io.StringIO() as buf, contextlib.redirect_stdout(buf):
                
                # 1. 파일 파싱
                parsed_result = parse_hwp5proc_xhtml(file_path)
                
                # 2. 파싱 결과를 바탕으로 청킹
                chunks = main_chunking(parsed_result)
                
            # 각 파일의 청크를 딕셔너리에 저장
            all_chunks[os.path.basename(file_path)] = chunks
                
        except Exception as e:
            print(f"파일 처리 중 오류 발생: {file_path}, 오류: {e}")
            continue
            
    print("\n\n모든 파일 처리 완료.")
    
    # 모든 파일의 청크 결과를 반환하거나, 필요에 따라 추가 작업을 수행할 수 있습니다.
    return all_chunks

# 메인 실행 코드
if __name__ == "__main__":
    # 파일들이 위치한 디렉토리 경로를 설정합니다.
    target_directory = '../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml'
    
    # 함수 실행
    all_processed_data = process_all_xhtml_files(target_directory)
    
    # 필요하다면 all_processed_data를 사용하여 다음 작업을 진행할 수 있습니다.
    # 예: print(f"처리된 파일 개수: {len(all_processed_data)}")

디렉토리 내 모든 XHTML 파일 처리 시작: ../data/processed/datapreprocessingbjs(hwp5proc)/all_xhtml
총 95개의 파일이 발견되었습니다.

[1/95] 파일 처리 중: 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.xhtml

[2/95] 파일 처리 중: 한국철도공사 (용역)_모바일오피스 시스템 고도화 용역(총체 및 1차).xhtml

[3/95] 파일 처리 중: 서민금융진흥원_서민금융진흥원 서민금융 채팅 상담시스템 구축.xhtml

[4/95] 파일 처리 중: 전북대학교_JST 공유대학(원) xAPI기반 LRS시스템 구축.xhtml

[5/95] 파일 처리 중: 한국어촌어항공단_한국어촌어항공단 경영관리시스템(ERP·GW) 기능 고도.xhtml

[6/95] 파일 처리 중: 광주과학기술원_실시간통합연구비관리시스템(RCMS)  연계 모듈 변경 사업.xhtml

[7/95] 파일 처리 중: 한국생산기술연구원_EIP3.0 고압가스 안전관리 시스템 구축 용역.xhtml

[8/95] 파일 처리 중: 파주도시관광공사_종량제봉투 판매관리 전산시스템 개선사업.xhtml

[9/95] 파일 처리 중: 한국수출입은행_(긴급) 모잠비크 마푸토 지능형교통시스템(ITS) 구축사업.xhtml

[10/95] 파일 처리 중: 울산광역시_2024년 버스정보시스템 확대 구축 및 기능개선 용역.xhtml

[11/95] 파일 처리 중: 재단법인 한국장애인문화예술원_2024년 장애인문화예술정보시스템 이음.xhtml

[12/95] 파일 처리 중: 국가철도공단_철도인프라 디지털트윈 정보화전략계획(ISP) 수립 용역(변.xhtml

[13/95] 파일 처리 중: 대검찰청_아태 사이버범죄 역량강화 허브(APC-HUB) 홈페이지 및 온라인 교.xhtml

[14/95] 파일 처리 중: 조선대학교_(재공고)2024 조선대학교 SW중심대학 사업관리시스템(WeHub) 구.xhtml

[15/95] 파일 처리 중: 한국농어촌공사_네팔 수자원

In [9]:
# 모든 파일 처리 결과 json 저장

# all_processed_data는 process_all_xhtml_files 함수가 반환한 결과입니다.
# all_processed_data = process_all_xhtml_files(target_directory)


output_dir = '../data/processed/datapreprocessingbjs(hwp5proc)'
output_file_path = os.path.join(output_dir, 'all_processed_data.json')

# 저장할 디렉터리가 없으면 생성합니다.
os.makedirs(output_dir, exist_ok=True)

try:
    with open(output_file_path, 'w', encoding='utf-8') as f:
        # 한글 깨짐을 방지하기 위해 ensure_ascii=False 옵션을 사용합니다.
        # indent=4는 가독성을 높여줍니다.
        json.dump(all_processed_data, f, ensure_ascii=False, indent=4)
    print(f"'{output_file_path}'에 처리된 데이터가 성공적으로 저장되었습니다. ✅")
except Exception as e:
    print(f"파일 저장 중 오류가 발생했습니다: {e}")


'../data/processed/datapreprocessingbjs(hwp5proc)/all_processed_data.json'에 처리된 데이터가 성공적으로 저장되었습니다. ✅


In [11]:
def load_chunks_from_json(file_path):
    """JSON 파일에서 청크 데이터를 불러옵니다."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        print(f"✅ {file_path}에서 데이터를 성공적으로 불러왔습니다.")
        return data
    except FileNotFoundError:
        print(f"❌ 오류: 파일을 찾을 수 없습니다: {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"❌ 오류: JSON 디코딩에 실패했습니다. 파일 형식을 확인해주세요.")
        return None

# JSON 파일 경로 설정 및 데이터 로드
json_file_path = '../data/processed/datapreprocessingbjs(hwp5proc)/all_processed_data.json'
all_processed_data = load_chunks_from_json(json_file_path)

if all_processed_data is None:
    # 데이터 로드 실패 시, 스크립트 종료
    exit()

# 모든 청크를 담을 단일 리스트 생성
chunks = []
for file_name, file_chunks in all_processed_data.items():
    for chunk in file_chunks:
        # 청크에 원본 파일 이름을 메타데이터로 추가
        chunk['metadata']['source_file'] = file_name
        chunks.append(chunk)

print(f"총 {len(chunks)}개의 청크가 최종적으로 준비되었습니다.")

✅ ../data/processed/datapreprocessingbjs(hwp5proc)/all_processed_data.json에서 데이터를 성공적으로 불러왔습니다.
총 22270개의 청크가 최종적으로 준비되었습니다.


In [12]:
print(type(all_processed_data))           # dict 예상
print(len(all_processed_data))            # 파일 개수 (95)
first_key = list(all_processed_data.keys())[0]
print("첫 번째 파일:", first_key)
print("샘플 청크:", all_processed_data[first_key][:2])

<class 'dict'>
95
첫 번째 파일: 국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.xhtml
샘플 청크: [{'type': 'table', 'subtype': 'general', 'content': '사 업 명: 주관기관, 국립인천해양박물관 해양자료관리시스템 구축 용역: 국립인천해양박물관', 'original_rows': 2, 'metadata': {'table_id': 1, 'source': 'table_extraction', 'priority': 4, 'source_file': '국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.xhtml'}, 'chunk_id': 'chunk_0000'}, {'type': 'table', 'subtype': 'general', 'content': ': 1차 사업, 사업 내용: 국립인천해양박물관 해양자료관리시스템 구축 및 초기 데이터 구축. : 2차 사업, 사업 내용: 리포팅툴 사용 S/W 및 리포트 출력양식 개발', 'original_rows': 3, 'metadata': {'table_id': 4, 'source': 'table_extraction', 'priority': 4, 'source_file': '국립인천해양박물관_국립인천해양박물관 해양자료관리시스템 구축 용.xhtml'}, 'chunk_id': 'chunk_0001'}]


In [13]:
# chunks 리스트를 정상적으로 순회하며 청크 ID 고유화
for chunk in tqdm(chunks, desc="청크 ID 고유화 및 임베딩 준비"):
    file_name = chunk['metadata'].get("source_file", "unknown_file")
    # 기존 chunk_id가 없으면 새로 생성
    original_id = chunk.get('chunk_id', f"chunk_{len(chunks):04d}")
    # 파일 이름과 원래 ID를 결합하여 고유 ID 생성
    chunk['chunk_id'] = f"{os.path.basename(file_name)}_{original_id}"

# ChromaDB와 임베딩 모델 관련 코드는 동일하게 사용 가능
db_path = "../data/processed/datapreprocessingbjs(hwp5proc)/chromaDB"
client = chromadb.PersistentClient(path=db_path)

# 임베딩 모델 지정
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
try:
    model = SentenceTransformer(model_name)
    print(f"✅ 임베딩 모델 '{model_name}'을 성공적으로 로드했습니다.")
except OSError as e:
    print(f"❌ 모델 로드 중 오류 발생: {e}")
    exit()

def get_or_create_collection(collection_name):
    try:
        collection = client.get_collection(name=collection_name)
        print(f"✅ 기존 컬렉션 '{collection_name}'을 불러왔습니다.")
        return collection
    except Exception:
        collection = client.create_collection(name=collection_name)
        print(f"✨ 새로운 컬렉션 '{collection_name}'을 생성했습니다.")
        return collection

# ChromaDB 컬렉션 설정
collection_name = "hwp_rag_collection_snunlp"
collection = get_or_create_collection(collection_name)

# # 기존 데이터가 있을 경우 삭제 (테스트용)
# if collection.count() > 0:
#     print(f"⚠️ 기존 데이터 {collection.count()}개를 삭제하고 다시 생성합니다.")
#     client.delete_collection(collection_name)
#     collection = client.create_collection(name=collection_name)

# 기존 컬렉션 초기화 (폴더 삭제 없이)
if collection.count() > 0:
    print(f"⚠️ 기존 데이터 {collection.count()}개를 삭제하고 컬렉션 초기화합니다.")
    # 컬렉션 안의 모든 문서 삭제
    ids_to_delete = [doc['id'] for doc in collection.peek(collection.count())]
    collection.delete(ids=ids_to_delete)

# 💡 이제 데이터 삽입 코드를 추가해야 합니다.
# 텍스트, ID, 메타데이터 리스트를 준비합니다.
documents = [chunk['content'] for chunk in chunks]
metadatas = [chunk['metadata'] for chunk in chunks]
ids = [chunk['chunk_id'] for chunk in chunks]

print(f"총 {len(ids)}개의 문서를 컬렉션에 추가합니다.")

# 청크 수가 많을 수 있으므로 배치로 나눠서 삽입
batch_size = 500
for i in tqdm(range(0, len(documents), batch_size), desc="데이터베이스에 문서 삽입"):
    batch_docs = documents[i:i + batch_size]
    batch_metadatas = metadatas[i:i + batch_size]
    batch_ids = ids[i:i + batch_size]
    
    collection.add(
        documents=batch_docs,
        metadatas=batch_metadatas,
        ids=batch_ids,
        embeddings=model.encode(batch_docs).tolist()
    )

print(f"✅ 모든 문서가 ChromaDB에 성공적으로 추가되었습니다. 현재 문서 수: {collection.count()}")

청크 ID 고유화 및 임베딩 준비: 100%|██████████| 22270/22270 [00:00<00:00, 520278.45it/s]
/opt/jhub-venv/lib/python3.10/site-packages/torch/cuda/__init__.py:734: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


✅ 임베딩 모델 'snunlp/KR-SBERT-V40K-klueNLI-augSTS'을 성공적으로 로드했습니다.
✨ 새로운 컬렉션 'hwp_rag_collection_snunlp'을 생성했습니다.
총 22270개의 문서를 컬렉션에 추가합니다.


데이터베이스에 문서 삽입: 100%|██████████| 45/45 [02:16<00:00,  3.03s/it]

✅ 모든 문서가 ChromaDB에 성공적으로 추가되었습니다. 현재 문서 수: 22270


In [2]:
# 현재 노트북 경로에 있는 .env 파일을 로드
load_dotenv("api_key.env")

# 환경 변수 읽기
openai_api_key = os.getenv("OPENAI_API_KEY")

print(openai_api_key[:5] + "..." if openai_api_key else "❌ API 키가 없습니다")

sk-pr...


In [11]:
# --- ChromaDB 컬렉션 로드 ---
db_path = "../data/processed/datapreprocessingbjs(hwp5proc)/chromaDB"
client = chromadb.PersistentClient(path=db_path)
collection_name = "hwp_rag_collection_snunlp"
collection = client.get_collection(name=collection_name)
print(f"✅ ChromaDB 컬렉션 '{collection_name}' 로드 완료")

# --- 임베딩 모델 로드 ---
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS"
model = SentenceTransformer(model_name)
print(f"✅ 임베딩 모델 '{model_name}' 로드 완료")

# --- OpenAI LLM 클라이언트 ---
openai_api_key = os.getenv("OPENAI_API_KEY")  # 환경 변수로 안전하게 관리
client_llm = OpenAI(api_key=openai_api_key)
print("✅ OpenAI LLM 클라이언트 초기화 완료")

def retrieval_augmented_generation(query: str, n_results=500):
    """RAG 방식으로 질문에 답변"""
    
    print("1️⃣ 검색(Retrieval) 시작...")
    query_embedding = model.encode([query]).tolist()
    
    # # ChromaDB에서 유사 문서 검색
    # results = collection.query(
    #     query_embeddings=query_embedding,
    #     n_results=n_results
    # )
    
    results = collection.query(
        query_embeddings=query_embedding,
        n_results=10,  # 1차적으로 헤더/제목 위주로 소수만 검색
        where={"subtype": "header"}
    )

    # 1차 검색 결과가 없으면 일반 검색
    if not results.get('documents', [[]])[0]:
        results = collection.query(
            query_embeddings=query_embedding,
            n_results=100 # 1차 검색 실패 시 일반 검색
        )
    
    retrieved_chunks = results.get('documents', [[]])[0]
    retrieved_metas = results.get('metadatas', [[]])[0]


    
    # 검색된 청크들을 메타데이터 우선순위에 따라 정렬하는 예시
    sorted_chunks = sorted(zip(retrieved_chunks, retrieved_metas), key=lambda x: x[1].get('priority', 0), reverse=True)
    
    # 상위 n개 청크만 선택하여 컨텍스트 구성
    context = ""
    for doc, meta in sorted_chunks[:10]: # 우선순위가 높은 상위 10개만 사용
        file_name = meta.get("source_file", "unknown")
        context += f"[출처: {file_name}]\n{doc}\n\n"



    
    if not retrieved_chunks:
        return "❌ 관련 문서를 찾지 못했습니다."
    
    print(f"✅ 검색 완료: {len(retrieved_chunks)}개의 문서 확보")
    
    # 검색 결과를 컨텍스트로 구성
    context = ""
    for doc, meta in zip(retrieved_chunks, retrieved_metas):
        file_name = meta.get("source_file", "unknown")
        context += f"[출처: {file_name}]\n{doc}\n\n"
    
    system_prompt = f"""
    당신은 문서 기반 AI 비서입니다.
    1) 주어진 문서만 참고하여 질문에 답변하세요.
    2) 질문과 관련된 모든 문서를 찾아 그 출처와 핵심 내용을 빠짐없이 요약해서 나열하세요.
    3) 만약 문서에 관련 정보가 없으면 '문서에 관련 정보가 없습니다.'라고 답변하세요.
    
    [검색 문서]
    {context}
    """
    
    print("2️⃣ 생성(Generation) 시작...")
    response = client_llm.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": query}
        ],
        temperature=0.7,
        max_tokens=700
    )
    
    return response.choices[0].message.content

✅ ChromaDB 컬렉션 'hwp_rag_collection_snunlp' 로드 완료
✅ 임베딩 모델 'snunlp/KR-SBERT-V40K-klueNLI-augSTS' 로드 완료
✅ OpenAI LLM 클라이언트 초기화 완료


In [12]:
# --- 사용 예시 ---
user_query = "'경기도'가 파일 이름에 들어가있는 제안요청서 알려줘."
answer = retrieval_augmented_generation(user_query)

print("\n--- 질문 ---")
print(user_query)
print("\n--- 답변 ---")
print(answer)

1️⃣ 검색(Retrieval) 시작...
✅ 검색 완료: 100개의 문서 확보
2️⃣ 생성(Generation) 시작...

--- 질문 ---
'경기도'가 파일 이름에 들어가있는 제안요청서 알려줘.

--- 답변 ---
다음 문서들이 '경기도'가 파일 이름에 포함된 제안요청서입니다:

1. [경기도사회서비스원_2024년 통합사회정보시스템 운영지원.xhtml]
   - 내용 요약: 보안서약서 제출, 기술적용계획표 작성, 사업관리자 서약서, 제안서 작성지침 등 포함.
   - 특징: 사업기간 내 기밀 유지, 보안위규 준수, 정량적 및 정성적 평가 등 안내.

2. [경기도 평택시_2024년도 평택시 버스정보시스템(BIS) 구축사업.xhtml]
   - 내용 요약: 제안서 제출과 평가에 관한 확약, 대표자 인감증명서 첨부, 사업자 보안예규 처리기준 포함.
   - 특징: 제안서 평가 및 계약 관련 확약서류 안내.

3. [경기도 안양시_호계체육관 배드민턴장 및 탁구장 예약시스템 구축 용역.xhtml]
   - 내용 요약: 입찰 참가신청서, 입찰보증금 납부 면제 및 지급 확약, 대리인 위임장, 인감 신고서, 서약서 등 제출서류 안내.
   - 특징: 인감 사용 및 대리인 위임 관련 지침 포함.

4. [재단법인경기도일자리재단_2025년 통합접수시스템 운영.xhtml]
   - 내용 요약: 사업관리자 서약, 보안 확약서, 투입인력 근태관리, 4대 사회보험 가입내역 등의 증빙서류 제출 요구.
   - 특징: 사업 운영과 관련된 인력 및 보안 사항 관리 지침 포함.

5. [경기도사회서비스원_2024년 통합사회정보시스템 운영지원.xhtml] (중복된 문서)
   - 내용 요약: 사업 제안서 작성 및 제출, 서약서, 일반현황, 조직 및 인력 현황 등 안내.

요약: 경기도 관련 제안요청서는 주로 사회서비스원, 평택시, 안양시, 경기도일자리재단 등 경기도 산하 기관 및 지자체의 시스템 구축 및 운영용역 관련이며, 제안서 작성, 보안서약, 입찰서류 제출, 평가기준 등이 상세

In [15]:
# --- 사용 예시 ---
user_query = "비용이 1억원 이하의 프로젝트 개수는 몇 개야? 각 프로젝트의 비용이 얼마인지도 함께 알려줘"
answer = retrieval_augmented_generation(user_query)

print("\n--- 질문 ---")
print(user_query)
print("\n--- 답변 ---")
print(answer)

1️⃣ 검색(Retrieval) 시작...
✅ 검색 완료: 100개의 문서 확보
2️⃣ 생성(Generation) 시작...

--- 질문 ---
비용이 1억원 이하의 프로젝트 개수는 몇 개야? 각 프로젝트의 비용이 얼마인지도 함께 알려줘

--- 답변 ---
주어진 문서에서 비용이 1억원 이하인 프로젝트 관련 정보는 다음과 같습니다.

1. [한국산업인력공단_RFID기반 국가자격 시험 결과물 스마트 관리시스템 도.xhtml]
   - 사업비: 80,000,000원 (8천만원, 부가세 10% 포함)
   - 내용: RFID를 활용한 시험 결과물 관리 시스템 구축 시범사업
   - 사업 기간: 계약일로부터 90일까지

이 외의 문서에서는 비용이 구체적으로 명시된 프로젝트가 없으므로, 비용이 1억원 이하인 프로젝트는 1건이며, 비용은 8천만원입니다.

요약:
- 총 1건
- 비용: 8천만원 (80,000,000원, 부가세 포함)
